In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV

import torch.utils.data as Data
import torch

In [3]:
data_train = np.load('Dataset/data_train.npy')
X = data_train[:,:3]
Y = data_train[:,-1]
print(X.shape)
print(Y.shape)

(2400000, 3)
(2400000,)


In [4]:
standarder = StandardScaler()
X_train , X_test, Y_train, Y_test = train_test_split(X,Y, test_size= 0.2, random_state=0)
standarder.fit(X_train)
X_train = standarder.transform(X_train)
X_test = standarder.transform(X_test)
X_train = torch.from_numpy(X_train)
#X_test = torch.from_numpy(X_test)
Y_train = torch.from_numpy(Y_train)
#Y_test = torch.from_numpy(Y_test)

In [5]:
data_train = Data.TensorDataset(X_train, Y_train)
#data_test = Data.TensorDataset(X_test, Y_test)

In [6]:
BATCH_SIZE = 1024
loder_train = Data.DataLoader(data_train, batch_size= BATCH_SIZE, shuffle= True)
#loder_test = Data.DataLoader(data_test, batch_size= BATCH_SIZE, shuffle= False)

In [7]:
clf = SVC(random_state= 0)

In [18]:
for step,(img, label) in enumerate(loder_train):
        
    param_grid = {
        'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 
        'degree':[2,3,4,5],
        'gamma': [0.001, 0.0001,0.01],
        'probability':[True,False]
    }    
    grid_search = GridSearchCV(clf, param_grid, n_jobs = 8, verbose=2, cv= 5)
    grid_search.fit(img.numpy(), label.numpy())
    
    break

Fitting 5 folds for each of 168 candidates, totalling 840 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  88 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 840 out of 840 | elapsed:    5.4s finished


In [19]:
model = grid_search.best_estimator_

In [20]:
grid_search.best_estimator_

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=True, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [21]:
model.score(X_test, Y_test)

0.8516666666666667

In [22]:
from sklearn.externals import joblib

D:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [25]:
joblib.dump(model, 'model/svm.joblib')

['model/svm.joblib']